<img src="images/logos/The-Construct-logo-new.png" width="700">

# PRESENTS...

# ROS Developers Open Class n.175

<img src="images/OpenClass175.jpeg" width="650" />

## How to Launch the Simulation

To start the simulation, first we'll need to source our workspace:

- Open a terminal window by clicking on the shell icon on the bottom left side of your screen:

<img src="images/shell-superapp.png" width="450" />

- Copy and paste the following terminal commands in your shell:


<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell
</span>

In [ ]:
ros2 launch the_construct_office_gazebo empty_ur3e.launch.xml

<img src="images/ur3-sim.png" />

<div class="jumbotron m-0">
    <hr />
    <h1 class="text-center">
        <span class="text-primary">
            Cartesian Path Planning in ROS2
        </span>
    </h1>
    <hr />
</div>

<div>
    <h1 class="text-center">
        <span class="text-primary">1.1</span>
        &nbsp;
        <span class="">What are Cartesian paths?</span>
    </h1>
</div>

In robot manipulation, a **Cartesian path** refers to a sequence of points in 3D space that define a trajectory for the end-effector of the robot in Cartesian coordinates (x, y, z). Each point along the path represents a specific position and orientation of the end-effector. Cartesian paths are defined in the robot's workspace and are represented as a series of waypoints through which the end-effector moves smoothly.

When planning a Cartesian path, the robot moves from one point to another in a straight line, following a smooth trajectory in Cartesian space. Cartesian paths are particularly useful for tasks where the end-effector needs to follow a specific trajectory, such as painting, welding, or 3D printing. They allow precise control over the end-effector's position and orientation throughout the movement, ensuring accuracy in complex tasks.

Let's set a practical example to better demonstrate this concept!

<div>
    <h1 class="text-center">
        <span class="text-primary">1.2</span>
        &nbsp;
        <span class="">Approach & Retreat</span>
    </h1>
</div>

In manipulation, when you want to pick an object from the environment, you don't send the end-effector directly to the pose  of the object, as the gripper might end up colliding with it. Instead, you send the end-effector to a pose **near** the object. Then, you execute the **approach** motion in order to get close enoguh to the object to pick it. Once you have picked the object, you execute the **retreat** motion to go back to the previous position. 

The full sequence would look something like this:

### 1. Move the arm to a pose near the object:

<img src="images/near-object.gif" width="500" />

### 2. Approach:

<img src="images/approach.gif" width="500" />

### 3. Retreat:

<img src="images/retreat.gif" width="500" />

Alright! So let's try to implement the **approach & retreat** sequence in the following example.

<div class="bg-success text-center">
    - Example -
</div>

Review the following code:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    approach_retreat.cpp
</span>

In [ ]:
#include <moveit/move_group_interface/move_group_interface.h>
#include <moveit/planning_scene_interface/planning_scene_interface.h>

#include <moveit_msgs/msg/display_robot_state.hpp>
#include <moveit_msgs/msg/display_trajectory.hpp>

static const rclcpp::Logger LOGGER = rclcpp::get_logger("move_group_demo");

int main(int argc, char **argv) {
  rclcpp::init(argc, argv);
  rclcpp::NodeOptions node_options;
  node_options.automatically_declare_parameters_from_overrides(true);
  auto move_group_node =
      rclcpp::Node::make_shared("move_group_interface_tutorial", node_options);

  rclcpp::executors::SingleThreadedExecutor executor;
  executor.add_node(move_group_node);
  std::thread([&executor]() { executor.spin(); }).detach();

  static const std::string PLANNING_GROUP_ARM = "ur_manipulator";

  moveit::planning_interface::MoveGroupInterface move_group_arm(
      move_group_node, PLANNING_GROUP_ARM);

  const moveit::core::JointModelGroup *joint_model_group_arm =
      move_group_arm.getCurrentState()->getJointModelGroup(PLANNING_GROUP_ARM);

  // Get Current State
  moveit::core::RobotStatePtr current_state_arm =
      move_group_arm.getCurrentState(10);

  std::vector<double> joint_group_positions_arm;
  current_state_arm->copyJointGroupPositions(joint_model_group_arm,
                                             joint_group_positions_arm);

  // Go Home
  move_group_arm.setStartStateToCurrentState();
  RCLCPP_INFO(LOGGER, "Going Home");

  // joint_group_positions_arm[0] = 0.00;  // Shoulder Pan
  joint_group_positions_arm[1] = -2.50; // Shoulder Lift
  joint_group_positions_arm[2] = 1.50;  // Elbow
  joint_group_positions_arm[3] = -1.50; // Wrist 1
  joint_group_positions_arm[4] = -1.55; // Wrist 2
  // joint_group_positions_arm[5] = 0.00;  // Wrist 3

  move_group_arm.setJointValueTarget(joint_group_positions_arm);

  moveit::planning_interface::MoveGroupInterface::Plan my_plan_arm;
  bool success_arm = (move_group_arm.plan(my_plan_arm) ==
                      moveit::core::MoveItErrorCode::SUCCESS);

  move_group_arm.execute(my_plan_arm);

  // Pregrasp

  RCLCPP_INFO(LOGGER, "Pregrasp Position");
  current_state_arm = move_group_arm.getCurrentState(10);
  current_state_arm->copyJointGroupPositions(joint_model_group_arm,
                                             joint_group_positions_arm);

  geometry_msgs::msg::Pose target_pose1;
  target_pose1.orientation.x = -1.0;
  target_pose1.orientation.y = 0.00;
  target_pose1.orientation.z = 0.00;
  target_pose1.orientation.w = 0.00;
  target_pose1.position.x = 0.343;
  target_pose1.position.y = 0.132;
  target_pose1.position.z = 0.264;
  move_group_arm.setPoseTarget(target_pose1);

  success_arm = (move_group_arm.plan(my_plan_arm) ==
                 moveit::core::MoveItErrorCode::SUCCESS);

  // Execute
  move_group_arm.execute(my_plan_arm);

  // Approach

  RCLCPP_INFO(LOGGER, "Approach to object");
  current_state_arm = move_group_arm.getCurrentState(10);
  current_state_arm->copyJointGroupPositions(joint_model_group_arm,
                                             joint_group_positions_arm);

  float delta = 0.04;
  target_pose1.orientation.x = -1.0;
  target_pose1.orientation.y = 0.00;
  target_pose1.orientation.z = 0.00;
  target_pose1.orientation.w = 0.00;
  target_pose1.position.x = 0.343;
  target_pose1.position.y = 0.132;
  target_pose1.position.z = target_pose1.position.z - delta;
  move_group_arm.setPoseTarget(target_pose1);

  success_arm = (move_group_arm.plan(my_plan_arm) ==
                 moveit::core::MoveItErrorCode::SUCCESS);

  // Execute
  move_group_arm.execute(my_plan_arm);

  // Retreat

  RCLCPP_INFO(LOGGER, "Retreat from object");
  current_state_arm = move_group_arm.getCurrentState(10);
  current_state_arm->copyJointGroupPositions(joint_model_group_arm,
                                             joint_group_positions_arm);

  target_pose1.orientation.x = -1.0;
  target_pose1.orientation.y = 0.00;
  target_pose1.orientation.z = 0.00;
  target_pose1.orientation.w = 0.00;
  target_pose1.position.x = 0.343;
  target_pose1.position.y = 0.132;
  target_pose1.position.z = target_pose1.position.z + delta;
  move_group_arm.setPoseTarget(target_pose1);

  success_arm = (move_group_arm.plan(my_plan_arm) ==
                 moveit::core::MoveItErrorCode::SUCCESS);

  // Execute
  move_group_arm.execute(my_plan_arm);

  rclcpp::shutdown();
  return 0;
}

You can run this program with the following commands.

1. First, launch the `move_group` node:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell #1
</span>

In [ ]:
ros2 launch test_moveit_config move_group.launch.py

2. Now, launch the previous code with the following command:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell #1
</span>

In [ ]:
ros2 launch moveit2_scripts approach_retreat.launch.py

Summarizing, the previous code uses the "typical" planning methods for solving the approach & retreat problem. So you  might be asking yourself right now.... Why is the arm performing all these complex motions when I just want it to move a little bit? And the answer is because of inverse kinematics (IK). Since you are setting a new end-effector pose, new IK solutions are computed, which result in overly complex motions.

So, the next question you are surely asking yourself is... can this behavior be changed in some way? And the answer is... YES! With **Cartesian Planning**!

Let's now have a look at the below code, that uses cartesian planning for achieveing this approach&retreat sequence.

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    cartesian_path.cpp
</span>

In [ ]:
#include <moveit/move_group_interface/move_group_interface.h>
#include <moveit/planning_scene_interface/planning_scene_interface.h>

#include <moveit_msgs/msg/display_robot_state.hpp>
#include <moveit_msgs/msg/display_trajectory.hpp>

static const rclcpp::Logger LOGGER = rclcpp::get_logger("move_group_demo");

int main(int argc, char **argv) {
  rclcpp::init(argc, argv);
  rclcpp::NodeOptions node_options;
  node_options.automatically_declare_parameters_from_overrides(true);
  auto move_group_node =
      rclcpp::Node::make_shared("move_group_interface_tutorial", node_options);

  rclcpp::executors::SingleThreadedExecutor executor;
  executor.add_node(move_group_node);
  std::thread([&executor]() { executor.spin(); }).detach();

  static const std::string PLANNING_GROUP_ARM = "ur_manipulator";

  moveit::planning_interface::MoveGroupInterface move_group_arm(
      move_group_node, PLANNING_GROUP_ARM);

  const moveit::core::JointModelGroup *joint_model_group_arm =
      move_group_arm.getCurrentState()->getJointModelGroup(PLANNING_GROUP_ARM);

  // Get Current State
  moveit::core::RobotStatePtr current_state_arm =
      move_group_arm.getCurrentState(10);

  std::vector<double> joint_group_positions_arm;
  current_state_arm->copyJointGroupPositions(joint_model_group_arm,
                                             joint_group_positions_arm);

  move_group_arm.setStartStateToCurrentState();

  // Go Home
  RCLCPP_INFO(LOGGER, "Going Home");

  // joint_group_positions_arm[0] = 0.00;  // Shoulder Pan
  joint_group_positions_arm[1] = -2.50; // Shoulder Lift
  joint_group_positions_arm[2] = 1.50;  // Elbow
  joint_group_positions_arm[3] = -1.50; // Wrist 1
  joint_group_positions_arm[4] = -1.55; // Wrist 2
  // joint_group_positions_arm[5] = 0.00;  // Wrist 3

  move_group_arm.setJointValueTarget(joint_group_positions_arm);

  moveit::planning_interface::MoveGroupInterface::Plan my_plan_arm;
  bool success_arm = (move_group_arm.plan(my_plan_arm) ==
                      moveit::core::MoveItErrorCode::SUCCESS);

  move_group_arm.execute(my_plan_arm);

  // Pregrasp
  RCLCPP_INFO(LOGGER, "Pregrasp Position");

  geometry_msgs::msg::Pose target_pose1;
  target_pose1.orientation.x = -1.0;
  target_pose1.orientation.y = 0.00;
  target_pose1.orientation.z = 0.00;
  target_pose1.orientation.w = 0.00;
  target_pose1.position.x = 0.343;
  target_pose1.position.y = 0.132;
  target_pose1.position.z = 0.264;
  move_group_arm.setPoseTarget(target_pose1);

  success_arm = (move_group_arm.plan(my_plan_arm) ==
                 moveit::core::MoveItErrorCode::SUCCESS);

  move_group_arm.execute(my_plan_arm);

  // Approach
  RCLCPP_INFO(LOGGER, "Approach to object!");

  std::vector<geometry_msgs::msg::Pose> approach_waypoints;
  target_pose1.position.z -= 0.03;
  approach_waypoints.push_back(target_pose1);

  target_pose1.position.z -= 0.03;
  approach_waypoints.push_back(target_pose1);

  moveit_msgs::msg::RobotTrajectory trajectory_approach;
  const double jump_threshold = 0.0;
  const double eef_step = 0.01;

  double fraction = move_group_arm.computeCartesianPath(
      approach_waypoints, eef_step, jump_threshold, trajectory_approach);

  move_group_arm.execute(trajectory_approach);

  // Retreat

  RCLCPP_INFO(LOGGER, "Retreat from object!");

  std::vector<geometry_msgs::msg::Pose> retreat_waypoints;
  target_pose1.position.z += 0.03;
  retreat_waypoints.push_back(target_pose1);

  target_pose1.position.z += 0.03;
  retreat_waypoints.push_back(target_pose1);

  moveit_msgs::msg::RobotTrajectory trajectory_retreat;

  fraction = move_group_arm.computeCartesianPath(
      retreat_waypoints, eef_step, jump_threshold, trajectory_retreat);

  move_group_arm.execute(trajectory_retreat);


###########################
  rclcpp::shutdown();
  return 0;
}

* **`approach_waypoints`**: Contains the waypoints to be followed by the end effector

* **`eef_step`**: Step size between end effector configurations of consecutive points in the result trajectory

* **`trajectory_retreat`**: Resulting cartesian trajectory

Now, to execute this program, run the following command:

<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell #1
</span>

In [ ]:
ros2 launch moveit2_scripts cartesian_path.launch.py

The result should look something like this:

<img src="images/cartesian-path.gif" width="600" />

<div class="bg-success text-center">
    - End of Example -
</div>

## Remember you can learn more about ROS2 Manipulation in our online course in the Academy:

<a href="https://app.theconstructsim.com/courses/167" target="_blank">ROS2 Manipulation Basics</a>

<a href="https://app.theconstructsim.com/courses/167" target="_blank"><img src="images/ros2-manip.png" width="" /></a>

# Build the robots of the future, Become a ROS DEVELOPER!